In [1]:
'''
Windows: Open Anaconda prompt
conda create --name tigre_env -c anaconda -c ccpi -c conda-forge  python tigre simpleitk ipykernel opencv astropy tomopy nibabel scikit-image scikit-learn scipy tqdm scikit-learn-intelex jupyter ipywidgets
conda activate tigre_env

conda list --export > conda-package-list.txt
conda create -n tigre_env --file conda-package-list.txt
'''

import json
import math
import multiprocessing
import os
import sys
from __future__ import division


import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import SimpleITK as sitk
import tomopy
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans
from PIL import Image
from scipy import interpolate
from scipy.ndimage import median_filter
from scipy.signal import medfilt2d
from skimage.registration import phase_cross_correlation
from tqdm import trange, tqdm
from typing import List

import tigre
import tigre.algorithms as algs
from tigre.utilities.geometry import Geometry

import shared_functions as s

kernel = Gaussian2DKernel(x_stddev=2)

drive = 'f:\\'
# basefolder = os.path.join(drive,'jasper','data','20220726_scanseries')
# base_folder = os.path.join(drive, 'jasper', 'data', '20220812_BreastTissueFFPE')
base_folder = os.path.join(drive, 'jasper', 'data', '20220819_ffpe_WhateverBreast')
base_json_file = os.path.join(base_folder, 'scan_settings.json')
results_folder = os.path.join(base_folder, 'results_fillgap')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)


# TODO Read whatever numbers possible from the relevant json file
'''
+ Open base_folder/Scan_settings.json
+ Get first key/value pair
+ Open the folder+filename from that pair
+ Get sample_z from that
'''



# Make a list of globals for the reconstruction setting, and log them in a json file
gReconParams = dict()

gReconParams['pixels'] = 512  # (pixels)
gReconParams['pixel_pitch'] = 0.055  # (mm)
gReconParams['fill_gap'] = True
gReconParams['median_filter'] = False
gReconParams['bad_pixel_correction'] = True
gReconParams['recon_voxels'] = (
    gReconParams['pixels'], gReconParams['pixels'], gReconParams['pixels'])  # number of voxels (vx)
gReconParams['recon_size'] = (gReconParams['pixels'] * gReconParams['pixel_pitch'], gReconParams['pixels']
                                * gReconParams['pixel_pitch'], gReconParams['pixels'] * gReconParams['pixel_pitch'])  # (mm)

''' TODO These should really be read from the JSON file! '''
gReconParams['distance_source_detector'] = 188.347  # 9+9+30+100+30+9+1.347 (mm)
gReconParams['z_stage_distance_mm'] = s.get_sample_z_from_first_scan_json(base_json_file) # Varies between 0 and 100 mm
gReconParams['distance_object_detector'] = 30 + \
    gReconParams['z_stage_distance_mm'] + 9+1.347  # (mm)
gReconParams['detector_rotation'] = (math.radians(-0.3), 0., 0.)  # (mm)

assert gReconParams['z_stage_distance_mm'] < 100 and gReconParams['z_stage_distance_mm'] > 0

print(gReconParams)

{'pixels': 512, 'pixel_pitch': 0.055, 'fill_gap': True, 'median_filter': False, 'bad_pixel_correction': True, 'recon_voxels': (512, 512, 512), 'recon_size': (28.16, 28.16, 28.16), 'distance_source_detector': 188.347, 'z_stage_distance_mm': 20.0, 'distance_object_detector': 60.347, 'detector_rotation': (-0.005235987755982988, 0.0, 0.0)}


In [2]:
if os.path.exists(base_json_file):
    f = open(base_json_file)
    dashboard = json.load(f)
    exp_time = []
    th0_list = []
    th1_list = []

    numpy_output_files = ['projs_stack_th0.npy', 'open_stack_th0.npy',
                          'projs_stack_th1.npy', 'open_stack_th1.npy', 'thlist_th0.npy', 'thlist_th1.npy']
    if s.files_exist(results_folder, numpy_output_files):
        print('Loading existing numpy files, should take ~7.5 seconds')

        spectral_projs_th0 = np.load(os.path.join(results_folder, numpy_output_files[0]))
        spectral_open_th0 = np.load(os.path.join(results_folder, numpy_output_files[1]))
        spectral_projs_th1 = np.load(os.path.join(results_folder, numpy_output_files[2]))
        spectral_open_th1 = np.load(os.path.join(results_folder, numpy_output_files[3]))
        th0_list = np.load(os.path.join(results_folder, numpy_output_files[4]))
        th1_list = np.load(os.path.join(results_folder, numpy_output_files[5]))

        for i in tqdm(dashboard['thresholdscan']):
            scan_folder = os.path.join(base_folder, dashboard['thresholdscan'][i]['projectionsfolder'])
            scan_json = os.path.join(scan_folder, dashboard['thresholdscan'][i]['projections_json'])
            open_image_folder = scan_folder
            open_image_json = scan_json
            folder_string = dashboard['thresholdscan'][i]['projectionsfolder']
            
            th0_keV = folder_string[0:folder_string.find('_')]
            th1_keV = folder_string[folder_string.find('_')+1:]

            angles = s.get_proj_angles(scan_json)
            z_offset = s.get_samplestage_z_offset(scan_json)
            detector_x_offsets, detector_y_offsets = s.get_detector_offsets(scan_json)
            exp_time.append(s.get_exposure_time_projection(scan_json))
        exp_time = np.asarray(exp_time)

    else:
        print(f'Making new numpy files, should take ~4.5 minutes. At least one file was missing :( ')
        
        spectral_projs_th0 = []
        spectral_open_th0 = []
        spectral_projs_th1 = []
        spectral_open_th1 = []
        th0_list = []
        th1_list = []
        th1_exp_time = []
        for i in tqdm(dashboard['thresholdscan']):
            scan_folder = os.path.join(
                base_folder, dashboard['thresholdscan'][i]['projectionsfolder'])
            scan_json = os.path.join(scan_folder, dashboard['thresholdscan'][i]['projections_json'])
            open_image_folder = scan_folder
            open_image_json = scan_json
            folder_string = dashboard['thresholdscan'][i]['projectionsfolder']
            th0_keV = folder_string[0:folder_string.find('_')]
            th1_keV = folder_string[folder_string.find('_')+1:]
            exp_time.append(s.get_exposure_time_projection(scan_json))
            th0_list.append(float(th0_keV))
            th1_list.append(float(th1_keV))

            projs_th0 = s.projectionsloader(
                scan_json, th0=True, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
            openimg_th0 = s.openimgloader(
                open_image_json, th0=True, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
            projs_th1 = s.projectionsloader(
                scan_json, th0=False, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
            openimg_th1 = s.openimgloader(
                open_image_json, th0=False, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
            spectral_projs_th0.append(projs_th0)
            spectral_open_th0.append(openimg_th0)
            spectral_projs_th1.append(projs_th1)
            spectral_open_th1.append(openimg_th1)
            detector_x_offsets, detector_y_offsets = s.get_detector_offsets(scan_json)
            angles = s.get_proj_angles(scan_json)
            z_offset = s.get_samplestage_z_offset(scan_json)
        spectral_projs_th0 = np.asarray(spectral_projs_th0)
        spectral_open_th0 = np.asarray(spectral_open_th0)
        spectral_projs_th1 = np.asarray(spectral_projs_th1)
        spectral_open_th1 = np.asarray(spectral_open_th1)
        exp_time = np.asarray(exp_time)
        th0_list = np.asarray(th0_list)
        th1_list = np.asarray(th1_list)

        np.save(os.path.join(results_folder, 'projs_stack_th0.npy'), spectral_projs_th0)
        np.save(os.path.join(results_folder, 'open_stack_th0.npy'), spectral_open_th0)
        np.save(os.path.join(results_folder, 'projs_stack_th1.npy'), spectral_projs_th1)
        np.save(os.path.join(results_folder, 'open_stack_th1.npy'), spectral_open_th1)
        np.save(os.path.join(results_folder, 'thlist_th0.npy'), th0_list)
        np.save(os.path.join(results_folder, 'thlist_th1.npy'), th1_list)

        # print(spectralprojs.shape, spectralopen.shape)
        # out = np.load(os.path.join(basefolder,'Projections_fitted.npy'))
        # openout = np.load(os.path.join(basefolder,'Projections_open_fitted.npy'))


Loading existing numpy files, should take ~7.5 seconds


100%|██████████| 8/8 [00:00<00:00, 23.79it/s]


In [ ]:
# print(spectral_projs_th0.shape)

# _global_detector_shift_y = s.find_optimal_offset(gReconParams, spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stage_offset=0, search_range=20)
# print(_global_detector_shift_y)
# global_detector_shift_y=0.12


In [3]:
# print(f'global_detector_shift_x = {global_detector_shift_x}')
# print(f'global_detector_shift_y = {global_detector_shift_y}')
print(spectral_open_th0.shape)  # E.g. (9, 32, 512, 512) (Thresholds, # of open images, pixels, pixels)

open_mean_th0 = np.mean(spectral_open_th0, axis=1)
open_mean_th1 = np.mean(spectral_open_th1, axis=1)

for i in range(open_mean_th0.shape[0]):
    # print(i, open_mean_th0.shape, open_mean_th0.shape[0])  # E.g. 0 (9, 512, 512) 9
    open_mean_th0[i, :, :] = open_mean_th0[i, :, :]/exp_time[i]
    open_mean_th1[i, :, :] = open_mean_th1[i, :, :]/exp_time[i]

for i in range(spectral_projs_th0.shape[0]):
    spectral_projs_th0[i, :, :, :] = spectral_projs_th0[i, :, :, :] / exp_time[i]
    spectral_projs_th1[i, :, :, :] = spectral_projs_th1[i, :, :, :] / exp_time[i]

# global_detector_shift_y = -0.31872500000000004

for th in range(0, open_mean_th0.shape[0]):
    a0 = 0
    a1 = (gReconParams['pixels'] // 2) - 1 # 255 for 512 pixels
    a2 = (gReconParams['pixels'] // 2) + 1 # 257 for 512 pixels
    a3 = gReconParams['pixels']
    mean = 0.25 * (
            np.mean(open_mean_th0[th, a0:a1, a0:a1]) +
            np.mean(open_mean_th0[th, a0:a1, a2:a3]) +
            np.mean(open_mean_th0[th, a2:a3, a0:a1]) +
            np.mean(open_mean_th0[th, a2:a3, a2:a3])
    )
    print(f'th = {th}, mean = {mean}')  # E.g. 36895.64768079413
    regression_out = np.zeros((3, gReconParams['pixels'], gReconParams['pixels']))
    DAC_values = np.zeros((gReconParams['pixels'], gReconParams['pixels']))

    print('Finding best DAC values per pixel...')
    DAC_values = s.save_and_or_load_npy_files(
            results_folder, f'th{th}_dac_values.npy', lambda: s.generate_dac_values(gReconParams, open_mean_th0, th0_list, mean, th))
    
    fit_array = spectral_projs_th0.reshape(spectral_projs_th0.shape[0], -1)

    print('Fitting polynomials... ~1 minute')
    regressions, res, _, _, _ = np.polyfit(th0_list, fit_array, 2, full=True)

    print('Calculating DACs...')
    calc_dacs = np.repeat(DAC_values[np.newaxis, :, :], spectral_projs_th0.shape[1], axis=0).flatten()

    print('Calculating projection data...')
    proj_data_flat = (regressions[0, :]*calc_dacs**2) + \
        (regressions[1, :]*calc_dacs**1) + regressions[2, :]
    projection_data = proj_data_flat.reshape(
        spectral_projs_th0.shape[1], spectral_projs_th0.shape[2], spectral_projs_th0.shape[3])
    s.save_array(results_folder, 'Projections_th0_' + str(th0_list[th])+'_keV.npy', projection_data)
    
    # TODO Ask Jasper about this VERY SUSPICIOUS CODE
    mean = (np.mean(projection_data[:, :, 0:10]) + np.mean(projection_data[:, :, 503:513]))/2
    ofc = -np.log(projection_data/mean)
    print(ofc.shape)
    s.save_array(results_folder, 'projs_th0_'+str(th0_list[th])+'OFC_interp.npy', ofc)

    ofc_bpc = s.save_and_or_load_npy_files(
            results_folder, f'th{th}_bpc.npy', lambda: s.generate_bad_pixel_corrected_array(ofc, gReconParams))
    ofc_bpc_mf = s.median_filter_projection_set(ofc_bpc, 3)
    if th == 0:
        print(f'th = {th}, finding optimal offset')
        
        centre_of_rotation_offset_x_mm = 2.51  # s.find_optimal_offset(gReconParams, spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stage_offset=0, search_range=25)
        centre_of_rotation_offset_y_mm = 0
        print(f'centre_of_rotation_offset_x_mm = {centre_of_rotation_offset_x_mm} (mm)')
        print(f'centre_of_rotation_offset_y_mm = {centre_of_rotation_offset_y_mm} (mm)')
        
        ni_img = nib.Nifti1Image(ofc_bpc_mf, np.eye(4))
        s.save_array(results_folder, 'Proj_th0_'+str(th0_list[th])+'OFC_BPC_MF.nii', ni_img)
    
    print('Doing recon finally!')
    img_th0 = s.recon_scan(gReconParams, ofc_bpc, angles, z_offset, detector_x_offsets,
                         detector_y_offsets, centre_of_rotation_offset_x_mm, centre_of_rotation_offset_y_mm)
    
    ni_img = nib.Nifti1Image(img_th0, np.eye(4))
    s.save_array(results_folder, 'Recon_th0_'+str(th0_list[th])+'OFC_BPC.nii', ni_img)

    img_th0 = s.recon_scan(gReconParams, ofc_bpc_mf, angles, z_offset, detector_x_offsets,
                         detector_y_offsets, centre_of_rotation_offset_y_mm)
    ni_img = nib.Nifti1Image(img_th0, np.eye(4))
    s.save_array(results_folder, 'Recon_th0_'+str(th0_list[th])+'OFC_BPC_MF.nii', ni_img)


(8, 64, 512, 512)
th = 0, mean = 30606.29922039497
Finding best DAC values per pixel...
Loading existing numpy file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\th0_dac_values.npy
Fitting polynomials... ~1 minute
Calculating DACs...
Calculating projection data...
Saving Numpy array file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\Projections_th0_5.0_keV.npy


C:\Users\Medipix3_Bois\AppData\Local\Temp\ipykernel_14968\3775556671.py:55: RuntimeWarning: invalid value encountered in log
  ofc = -np.log(projection_data/mean)


(720, 512, 512)
Saving Numpy array file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\projs_th0_5.0OFC_interp.npy
Loading existing numpy file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\th0_bpc.npy
th = 0, finding optimal offset
global_detector_shift_y = 0.21 (mm)
Saving Nifti array file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\Proj_th0_5.0OFC_BPC_MF.nii
Doing recon finally!
Saving Nifti array file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\Recon_th0_5.0OFC_BPC.nii
Saving Nifti array file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\Recon_th0_5.0OFC_BPC_MF.nii
th = 1, mean = 25198.978491218284
Finding best DAC values per pixel...
Loading existing numpy file: f:\jasper\data\20220819_ffpe_WhateverBreast\results_fillgap\th1_dac_values.npy
Fitting polynomials... ~1 minute
Calculating DACs...
Calculating projection data...


KeyboardInterrupt: 

In [ ]:
print(f'th = {th}, finding optimal offset')
global_detector_shift_y = .2 #find_optimal_offset(spectral_projs_th0[1, :, :, :], angles, detector_x_offsets, detector_y_offsets, stageoffset=0, searchrange=25)
print(f'global_detector_shift_y = {global_detector_shift_y} (mm)')

ni_img = nib.Nifti1Image(ofc_bpc_mf, np.eye(4))
s.save_array(results_folder, 'Proj_th0_'+str(th0_list[th])+'OFC_BPC_MF.nii', ni_img)

print('Doing recon finally!')
img_th0 = s.recon_scan(gReconParams, ofc_bpc, angles, z_offset, detector_x_offsets,
                    detector_y_offsets, global_detector_shift_y)

ni_img = nib.Nifti1Image(img_th0, np.eye(4))
s.save_array(results_folder, 'Recon_th0_'+str(th0_list[th])+'OFC_BPC.nii', ni_img)

img_th0 = s.recon_scan(gReconParams, ofc_bpc_mf, angles, z_offset, detector_x_offsets,
                    detector_y_offsets, global_detector_shift_y)
ni_img = nib.Nifti1Image(img_th0, np.eye(4))
s.save_array(results_folder, 'Recon_th0_'+str(th0_list[th])+'OFC_BPC_MF.nii', ni_img)


In [ ]:
for th1 in range(0, open_mean_th1.shape[0]):
    mean = 0.25*(np.mean(open_mean_th1[th1, 0:255, 0:255]) + np.mean(open_mean_th1[th1, 0:255, 257:512]) + np.mean(
        open_mean_th1[th1, 257:512, 0:255]) + np.mean(open_mean_th1[th1, 257:512, 257:512]))
    print(mean)
    regression_out = np.zeros((3, 512, 512))
    DACvalues = np.zeros((512, 512))
    for i in range(0, open_mean_th1.shape[1]):
        for j in range(0, open_mean_th1.shape[2]):
            yvalues = open_mean_th1[:, i, j]
            regressions, res, _, _, _ = np.polyfit(th1_list, yvalues, 2, full=True)
            regression_out[:, i, j] = regressions
            DAC = s.solve_for_y(regressions, mean)[1]
            if (DAC > th1_list[th1]*2) or (DAC < th1_list[th1]/2):
                DAC = th1_list[th1]
            DACvalues[i, j] = DAC
    fitarray = spectral_projs_th1.reshape(spectral_projs_th1.shape[0], -1)
    regressions, res, _, _, _ = np.polyfit(th1_list, fitarray, 2, full=True)
    calcdacs = np.repeat(DACvalues[np.newaxis, :, :], spectral_projs_th1.shape[1], axis=0).flatten()

    projdataflat = (regressions[0, :]*calcdacs**2) + \
        (regressions[1, :]*calcdacs**1) + regressions[2, :]
    projectiondata = projdataflat.reshape(
        spectral_projs_th1.shape[1], spectral_projs_th1.shape[2], spectral_projs_th1.shape[3])
    np.save(os.path.join(results_folder, 'Projections_th1_'
            + str(th1_list[th1])+'_keV.npy'), projectiondata)

    mean = (np.mean(projectiondata[:, :, 0:10]) + np.mean(projectiondata[:, :, 503:513]))/2
    ofc = -np.log(projectiondata/mean)
    ni_img = nib.Nifti1Image(ofc, np.eye(4))
    np.save(os.path.join(results_folder, 'projs_th1_'+str(th1_list[th1])+'OFC_interp.npy'), ofc)
    ofc_mf = s.median_filter_projection_set(ofc, 5)
    diff_mf = np.abs(ofc-ofc_mf)
    meanmap = np.mean(diff_mf, axis=0)
    stdmap = np.std(diff_mf, axis=0)
    badmap = np.ones((512, 512))
    half = np.int32(badmap.shape[0]/2)
    badmap[half-2:half+1] = 0
    badmap[:, half-2:half+1] = 0
    badmap[meanmap > 0.2] = 0
    badmap[stdmap > 0.05] = 0
    ofc_bpc = s.apply_badmap_to_projections(ofc, badmap)
    ofc_bpc_mf = s.median_filter_projection_set(ofc_bpc, 3)
    img_th1 = s.recon_scan(ofc_bpc, angles, z_offset, detector_x_offsets,
                         detector_y_offsets, global_detector_shift_y)
    ni_img = nib.Nifti1Image(img_th1, np.eye(4))
    nib.save(ni_img, os.path.join(results_folder, 'Recon_th1_'+str(th1_list[th1])+'OFC_BPC.nii'))
    img_th1 = s.recon_scan(ofc_bpc_mf, angles, z_offset, detector_x_offsets,
                         detector_y_offsets, global_detector_shift_y)
    ni_img = nib.Nifti1Image(img_th1, np.eye(4))
    nib.save(ni_img, os.path.join(results_folder, 'Recon_th1_'+str(th1_list[th1])+'OFC_BPC_MF.nii'))


In [ ]:

openmeansingle_th0 = np.mean(open_mean_th0, axis=(1, 2))
openmeansingle_th1 = np.mean(open_mean_th1, axis=(1, 2))
plt.scatter(th0_list, openmeansingle_th0, label="th0")
plt.scatter(th1_list, openmeansingle_th1, label="th1")


In [ ]:
# global_detector_shift_y = 0.37565
global_detector_shift_y = -0.31872500000000004

hyperlist = [500]
lambdalist = [0.010]
for hyper in hyperlist:
    for tvlambda in lambdalist:
        # hyper=2.0e3
        tviter = 100
        # tvlambda=0.005
        img_th0 = s.recon_scan_fista(ofc_2_mf.astype(np.float32), angles, z_offset, detector_x_offsets,
                                   detector_y_offsets, global_detector_shift_y, hyper, tviter, tvlambda)
        ni_img = nib.Nifti1Image(img_th0, np.eye(4))
        nib.save(ni_img, os.path.join(base_folder, 'recon_fista_mf_'
                 + str(tviter) + '_' + str(hyper)+'_'+str(tvlambda) + '.nii'))


In [ ]:
drive = 'f:\\'
# basefolder = os.path.join(drive,'jasper','data','20220726_scanseries')
base_folder = os.path.join(drive, 'jasper', 'data', '20220805_tumourWhateverBreast')
base_json_file = os.path.join(base_folder, 'scan_settings.json')
results_folder = os.path.join(base_folder, 'results_open_after')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)
if os.path.exists(base_json_file):
    f = open(base_json_file)
    dashboard = json.load(f)
    spectral_projs_th0 = []
    spectral_open_th0 = []
    spectral_projs_th1 = []
    spectral_open_th1 = []
    th0_list = []
    exp_time = []
    th1_list = []
    th1_exp_time = []
    for i in dashboard['thresholdscan']:
        scan_folder = os.path.join(base_folder, dashboard['thresholdscan'][i]['projectionsfolder'])
        scan_json = os.path.join(scan_folder, dashboard['thresholdscan'][i]['projections_json'])
        open_image_folder = os.path.join(
            base_folder, dashboard['thresholdscan'][i]['openimagesfolder'])
        open_image_json = os.path.join(
            open_image_folder, dashboard['thresholdscan'][i]['openimages_json'])
        folder_string = dashboard['thresholdscan'][i]['projectionsfolder']
        th0_keV = folder_string[0:folder_string.find('_')]
        th1_keV = folder_string[folder_string.find('_')+1:]
        exp_time.append(s.get_exposure_time_projection(scan_json))
        th0_list.append(float(th0_keV))
        th1_list.append(float(th1_keV))

        projs_th0 = s.projectionsloader(
            scan_json, th0=True, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
        openimg_th0 = s.openimgloader(
            open_image_json, th0=True, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
        projs_th1 = s.projectionsloader(
            scan_json, th0=False, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
        openimg_th1 = s.openimgloader(
            open_image_json, th0=False, badpixelcorr=gReconParams['bad_pixel_correction'], medianfilter=gReconParams['median_filter'], fillgap=gReconParams['fill_gap'])
        spectral_projs_th0.append(projs_th0)
        spectral_open_th0.append(openimg_th0)
        spectral_projs_th1.append(projs_th1)
        spectral_open_th1.append(openimg_th1)
        detector_x_offsets, detector_y_offsets = s.get_detector_offsets(scan_json)
        angles = s.get_proj_angles(scan_json)
        z_offset = s.get_samplestage_z_offset(scan_json)
    spectral_projs_th0 = np.asarray(spectral_projs_th0)
    spectral_open_th0 = np.asarray(spectral_open_th0)
    spectral_projs_th1 = np.asarray(spectral_projs_th1)
    spectral_open_th1 = np.asarray(spectral_open_th1)
    exp_time = np.asarray(exp_time)
    th0_list = np.asarray(th0_list)
    th1_list = np.asarray(th1_list)

    np.save(os.path.join(results_folder, 'projs_stack_th0.npy'), spectral_projs_th0)
    np.save(os.path.join(results_folder, 'open_stack_th0.npy'), spectral_open_th0)
    np.save(os.path.join(results_folder, 'projs_stack_th1.npy'), spectral_projs_th1)
    np.save(os.path.join(results_folder, 'open_stack_th1.npy'), spectral_open_th1)
    np.save(os.path.join(results_folder, 'thlist_th0.npy'), th0_list)
    np.save(os.path.join(results_folder, 'thlist_th1.npy'), th1_list)

    # print(spectralprojs.shape, spectralopen.shape)
    # out = np.load(os.path.join(basefolder,'Projections_fitted.npy'))
    # openout = np.load(os.path.join(basefolder,'Projections_open_fitted.npy'))


In [ ]:
drive = 'f:\\'
basefolder = os.path.join(drive, 'jasper', 'data', '20220603_RealTumor')
basejsonfile = os.path.join(basefolder, 'scan_settings.json')

if os.path.exists(basejsonfile):
    f = open(basejsonfile)
    dashboard = json.load(f)
    for i in dashboard['thresholdscan']:
        scanfolder = os.path.join(basefolder, dashboard['thresholdscan'][i]['projectionsfolder'])
        scanjson = os.path.join(scanfolder, dashboard['thresholdscan'][i]['projections_json'])
        detector_x_offsets, detector_y_offsets = s.get_detector_offsets(scanjson)
        angles = s.get_proj_angles(scanjson)
        z_offset = s.get_samplestage_z_offset(scanjson)
        projs_th0 = s.get_OFC_projections(scanjson, th0=True, UseOrig=False, ApplyLog=True)

        # projectionsCorr = tomopy.prep.stripe.remove_all_stripe(projs_th0, snr=3, la_size=61, sm_size=21, dim=1, ncore=60, nchunk=None)
        projectionsCorr = tomopy.prep.stripe.remove_stripe_fw(
            projs_th0, level=None, wname='db5', sigma=0.2, pad=True, ncore=60, nchunk=None)
        # global_detector_shift_y = find_optimal_offset(projs_th0, angles, detector_x_offsets, detector_y_offsets, z_offset)
        global_detector_shift_y = 0.1915
        gReconVoxels = (600, 512, 512)             # number of voxels              (vx)
        gReconSize = (15, 12.8, 12.8)          # total size of the image       (mm)

        img_th0 = s.recon_scan(projs_th0, angles, z_offset, detector_x_offsets,
                             detector_y_offsets, global_detector_shift_y)
        ni_img = nib.Nifti1Image(img_th0, np.eye(4))
        nib.save(ni_img, os.path.join(basefolder, 'recon_'+str(0)+'_th0_orig.nii'))
        # imgfdk = tomopy.misc.corr.remove_ring(img_th0, center_x=None, center_y=None, thresh=1000.0, thresh_max=5000.0, thresh_min=10.0, theta_min=50, rwidth=10, int_mode='WRAP', ncore=60, nchunk=None, out=None)
        # ni_img = nib.Nifti1Image(imgfdk, np.eye(4))
        # nib.save(ni_img,  os.path.join(basefolder,'recon_'+str(0)+'_th0_corr_pre_post.nii'))
        img_th0 = s.recon_scan(projectionsCorr, angles, z_offset, detector_x_offsets,
                             detector_y_offsets, global_detector_shift_y)
        ni_img = nib.Nifti1Image(img_th0, np.eye(4))
        nib.save(ni_img, os.path.join(basefolder, 'recon_'+str(0)+'_th0_corr2.nii'))
